In [1]:
from sklearn.preprocessing import StandardScaler
from sail.models.torch.tcn import TCNRegressor
import numpy as np
from sklearn.datasets import make_regression

In [2]:
# Create data as we do in tests
def regression_data(n_samples, n_features):
    X, y = make_regression(  # type: ignore
        n_samples, n_features, n_informative=10, bias=0, random_state=0
    )
    X, y = X.astype(np.float32), y.astype(np.float32).reshape(-1, 1)
    Xt = StandardScaler().fit_transform(X)
    yt = StandardScaler().fit_transform(y)
    return Xt, yt


In [3]:
# Set size of data
from tabnanny import verbose


n_samples = 1000
n_features = 12
batch_size = 20

# Get regression data
X, y = regression_data(n_samples, n_features)

# Add dummy axis, as net requires it
X = X[:, :, np.newaxis]

# Initialize network as TCNRegressor
learner_tcn = TCNRegressor(n_features, 1)

# Calculate epochs as even divisor of n_samples and batch_size for sake of easiness
epochs = n_samples // batch_size
for i in range(epochs):
    # Extract X and y values of batch_size from training set
    X_step = X[i * batch_size : (i + 1) * batch_size, :, :]
    y_step = y[i * batch_size : (i + 1) * batch_size]

    # Run partial fit
    learner_tcn.partial_fit(X_step, y_step)

# Run prediction (note: this is of course the data that we trained on,
# but only for convenience sake; we are just testing if partial_fit works)
y_prediction = learner_tcn.predict(X)


  epoch    train_loss     dur
-------  ------------  ------
      1        1.5330  0.1264
      2        1.6742  0.0292
      3        1.5040  0.0683
      4        2.0833  0.0310
      5        1.0204  0.0287
      6        0.9228  0.0285
      7        1.6015  0.0318
      8        1.7149  0.0274
      9        1.5731  0.0263
     10        1.4039  0.0259
     11        1.3001  0.0298
     12        0.8351  0.0320
     13        0.5619  0.0268
     14        1.9845  0.0270
     15        0.7550  0.0281
     16        1.1101  0.0203
     17        0.9978  0.0255
     18        0.8824  0.0249
     19        0.8391  0.0238
     20        1.4615  0.0217
     21        0.7344  0.0249
     22        0.9351  0.0216
     23        0.8725  0.0211
     24        1.8060  0.0244
     25        0.7775  0.0296
     26        0.6300  0.0310
     27        1.2439  0.0335
     28        0.7871  0.0288
     29        0.9565  0.0263
     30        1.0520  0.0209
     31        0.7190  0.0190
     32   

In [4]:
# Plot the predictions and the truth
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "vscode"

y_pred = y_prediction.flatten().tolist()
y_true = y.flatten().tolist()
x = np.arange(len(y_true))
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=x, y=y_pred, name="predicted", line=dict(color="red", width=2))
)
fig.add_trace(
    go.Scatter(x=x, y=y_true, name="actual", line=dict(color="blue", width=2))
)
fig.show()
